#### This code is to explore the relationship between pathway from website and reaction in Neo4j

In [2]:
from neo4j import GraphDatabase
import py2neo
import pandas as pd
import importlib
import progressbar
import numpy as np

In [3]:
# Initialize connection to database
driver = GraphDatabase.driver('bolt://localhost:7687', 
                              auth=('neo4j', '19992005zwj'), 
                              encrypted=False)

In [4]:
dic_pathway = np.load('./dicCVD_pathway_neo4j.npy', allow_pickle=True).item()

### About pathway from website

#### Relationship with reaction

In [35]:
reac = []
path_reac = {}
for i in dic_pathway:
    for j in dic_pathway[i]:
        path_reac[j] = []
        query = 'MATCH (n:Pathway) WHERE n.stId =\'' + j +\
                '\' WITH n MATCH (n:Pathway) -[r]- (m:ReactionLikeEvent) RETURN m.stId'
        with driver.session() as session:
            info = session.run(query)
            for item in info:
                path_reac[j].append(item.values()[0])
                if item.values()[0] not in reac:
                    reac.append(item.values()[0])
                   

In [36]:
path_reac

{'R-HSA-9008059': ['R-HSA-9008694',
  'R-HSA-9009910',
  'R-HSA-9008692',
  'R-HSA-9009072',
  'R-HSA-9012689',
  'R-HSA-9008894',
  'R-HSA-9012542',
  'R-HSA-9008696',
  'R-HSA-9008928',
  'R-HSA-9008577',
  'R-HSA-8848335',
  'R-HSA-9008684',
  'R-HSA-9012556',
  'R-HSA-9008587'],
 'R-HSA-9029569': ['R-HSA-9035279',
  'R-HSA-9024334',
  'R-HSA-9618394',
  'R-HSA-9618392',
  'R-HSA-9024386',
  'R-HSA-9035143',
  'R-HSA-9024326',
  'R-HSA-9624925',
  'R-HSA-9031512',
  'R-HSA-9029531',
  'R-HSA-9605057',
  'R-HSA-9035167',
  'R-HSA-9031518',
  'R-HSA-9029521',
  'R-HSA-9035281',
  'R-HSA-9038545',
  'R-HSA-9029591',
  'R-HSA-9031510',
  'R-HSA-9024361',
  'R-HSA-9618486',
  'R-HSA-9035180',
  'R-HSA-9029551',
  'R-HSA-9035169',
  'R-HSA-9031521',
  'R-HSA-9657786',
  'R-HSA-9031522',
  'R-HSA-9657791',
  'R-HSA-9031527',
  'R-HSA-9605052',
  'R-HSA-9029561',
  'R-HSA-9624353',
  'R-HSA-9035133',
  'R-HSA-9657767',
  'R-HSA-9618407',
  'R-HSA-9618405',
  'R-HSA-9035185',
  'R-HSA-902955

In [5]:
stat = {}
for i in path_reac:
    for j in path_reac[i]:
        temp = j
        if temp not in stat:
            stat[temp] = 1
        else:
            stat[temp] += 1
len(stat)

1277

### Find the preceding-reaction of each pathway

In [39]:
query = 'MATCH (n:Pathway) WHERE n.stId = \'R-HSA-9008059\''+\
        ' WITH n MATCH (n) -[r]- (m:ReactionLikeEvent)'+\
        'WITH m MATCH (m) -[t:precedingEvent]-> (u) RETURN m.stId,t,u.stId'
with driver.session() as session:
    info = session.run(query)
    pre = []
    lat = []
    num = []
    for item in info:
        pre.append(item.values()[0])
        lat.append(item.values()[2])
        #print(item.values()[0],item.values()[2])
    #for i in range(len(pre)):
    #    if lat[i] not in pre:
    #        pre[i] = 'null'
    #        lat[i] = 'null'

for i in range(len(pre)):
    print(pre[i], ' ', lat[i])

R-HSA-9008694   R-HSA-9012689
R-HSA-9009910   R-HSA-9012689
R-HSA-9008692   R-HSA-9009910
R-HSA-9008692   R-HSA-9012689
R-HSA-9009072   R-HSA-9008577
R-HSA-9012689   R-HSA-9012556
R-HSA-9008894   R-HSA-9008928
R-HSA-9012542   R-HSA-9012689
R-HSA-9008696   R-HSA-9012689
R-HSA-9008928   R-HSA-9009910
R-HSA-9008577   R-HSA-8848335
R-HSA-8848335   R-HSA-9008694
R-HSA-9008684   R-HSA-9008577
R-HSA-9012556   R-HSA-9012542
R-HSA-9008587   R-HSA-9008694


In [6]:
stat = pd.DataFrame.from_dict(stat, orient = 'index', columns=['numbers'])
stat = stat.reset_index().rename(columns = {'index':'Name'})

In [7]:
stat.sort_values('numbers',ascending=False).head()

,Name,numbers
878,R-HSA-5693609,2
1107,R-HSA-3239014,2
906,R-HSA-266082,2
453,R-HSA-71682,2
296,R-HSA-9646295,2


In [65]:
stat.to_csv('./pathway_reaction.csv')

#### Total number of reaction related to pathway in Neo4j

In [66]:
len(reac)

1277

In [67]:
dic_reaction = np.load('./dicCVD_reaction_neo4j.npy', allow_pickle=True).item()

In [68]:
tp = []
for i in dic_reaction:
    for j in dic_reaction[i]:
        if j not in tp:
            tp.append(j)

#### Total number of reaction from website

In [69]:
len(tp)

384

In [70]:
c = 0
for i in reac:
    if i in tp:
        c += 1
    

#### The number of the overlapped reaction

In [71]:
c

124

#### Relationship with Bioprocess

In [11]:
process = []
count = 0
path_process = {}
for i in dic_pathway:
    for j in dic_pathway[i]:
        path_process[j] = []
        query = 'MATCH (n:Pathway) WHERE n.stId =\'' + j +\
                '\' WITH n MATCH (n:Pathway) -[r]- (m:GO_BiologicalProcess) RETURN m.displayName'
        #print(query)
        with driver.session() as session:
            info = session.run(query)
            for item in info:
                path_process[j].append(item.values()[0])
                count += 1

In [12]:
stat = {}
for i in path_process:
    for j in path_process[i]:
        temp = j
        if temp not in stat:
            stat[temp] = 1
        else:
            stat[temp] += 1
len(stat)

49

In [13]:
stat = pd.DataFrame.from_dict(stat, orient = 'index', columns=['numbers'])
stat = stat.reset_index().rename(columns = {'index':'Name'})

In [14]:
stat.sort_values('numbers',ascending=False).head()

,Name,numbers
10,Notch signaling pathway,3
1,long-chain fatty acid biosynthetic process,2
9,regulation of transcription by RNA polymerase II,2
0,cellular response to cytokine stimulus,1
37,cellular response to stress,1


#### Relationship with CellularComponet

In [15]:
count = 0
path_cell = {}
for i in dic_pathway:
    for j in dic_pathway[i]:
        path_cell[j] = []
        query = 'MATCH (n:Pathway) WHERE n.stId =\'' + j +\
                '\' WITH n MATCH (n:Pathway) -[r]- (m:GO_CellularComponent) RETURN m.displayName'
        #print(query)
        with driver.session() as session:
            info = session.run(query)
            for item in info:
                path_cell[j].append(item.values()[0])
                count += 1

In [16]:
stat = {}
for i in path_cell:
    for j in path_cell[i]:
        temp = j
        if temp not in stat:
            stat[temp] = 1
        else:
            stat[temp] += 1
len(stat)

12

In [17]:
stat = pd.DataFrame.from_dict(stat, orient = 'index', columns=['numbers'])
stat = stat.reset_index().rename(columns = {'index':'Name'})

In [18]:
stat.sort_values('numbers',ascending=False).head()

,Name,numbers
2,cytosol,23
0,plasma membrane,17
3,nucleoplasm,17
1,extracellular region,13
4,endoplasmic reticulum membrane,3


#### Relationship with Diseases

In [19]:
dis = []
count = 0
path_dis = {}
for i in dic_pathway:
    for j in dic_pathway[i]:
        path_dis[j] = []
        query = 'MATCH (n:Pathway) WHERE n.stId =\'' + j +\
                '\' WITH n MATCH (n:Pathway) -[r]- (m:Disease) RETURN m.displayName'
        #print(query)
        with driver.session() as session:
            info = session.run(query)
            for item in info:
                path_dis[j].append(item.values()[0])
                count += 1

In [20]:
stat = {}
for i in path_dis:
    for j in path_dis[i]:
        temp = j
        if temp not in stat:
            stat[temp] = 1
        else:
            stat[temp] += 1
len(stat)

64

In [21]:
stat = pd.DataFrame.from_dict(stat, orient = 'index', columns=['numbers'])
stat = stat.reset_index().rename(columns = {'index':'Name'})

In [22]:
stat.sort_values('numbers',ascending=False).head()

,Name,numbers
15,cancer,21
17,bone development disease,7
22,primary immunodeficiency disease,6
56,Ehlers-Danlos syndrome,3
40,renal tubular acidosis,2


#### Relationship with Protein

In [44]:
pro = []
count = 0
path_pro = {}
for i in dic_pathway:
    for j in dic_pathway[i]:
        path_pro[j] = []
        query = 'MATCH (n:Pathway) WHERE n.stId =\'' + j +\
                '\' WITH n MATCH (n:Pathway) -[r]- (m:ReferenceEntity) RETURN m.displayName'
        #print(query)
        with driver.session() as session:
            info = session.run(query)
            for item in info:
                path_pro[j].append(item.values()[0])
                count += 1

In [45]:
stat = {}
for i in path_pro:
    for j in path_pro[i]:
        temp = j
        if temp not in stat:
            stat[temp] = 1
        else:
            stat[temp] += 1
len(stat)

0

### About reaction from website

#### relationship with pathway

In [23]:
path = []
for i in dic_reaction:
    for j in dic_reaction[i]:
        query = 'MATCH (n:Reaction) WHERE n.stId =\'' + j +\
                '\' WITH n MATCH (n) -[r]- (m:Pathway) RETURN m.stId'
        with driver.session() as session:
            info = session.run(query)
            for item in info:
                #print(item.values())
                if item.values()[0] not in path:
                    path.append(item.values()[0])

len(path)

200

In [24]:
tp = []
for i in dic_pathway:
    for j in dic_pathway[i]:
        if j not in tp:
            tp.append(j)

In [25]:
len(tp)

209

In [26]:
c = 0
for i in path:
    if i in tp:
        c += 1

#### The number of the overlapped pathway

In [27]:
c

46